<a href="https://colab.research.google.com/github/mahalakshmishettysagar/Hello-world/blob/master/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow


In [0]:
!pip install keras-tuner

In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [0]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [0]:
print(tf.__version__)

In [0]:
fashion_mnist=keras.datasets.fashion_mnist

In [0]:

(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

In [0]:
train_images = train_images/255.0
test_images = test_images/255.0


In [0]:
train_images[0].shape


In [0]:
train_images =train_images.reshape(len(train_images),28,28,1)
test_images =test_images.reshape(len(test_images),28,28,1)

In [0]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [0]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [0]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [0]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

In [0]:
model=tuner_search.get_best_models(num_models=1)[0]

In [0]:


model.summary()


In [0]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

In [0]:
model.save("model.h5")
print("Saved model to disk")